In [5]:
d

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
3,Machine Learning,Distinct Types of ML,Supervised Learning,Supervised learning trains a model using label...
4,Machine Learning,Distinct Types of ML,Unsupervised Learning,Unsupervised learning works with unlabeled dat...
5,Machine Learning,Distinct Types of ML,Semisupervised Learning,Semi-supervised learning combines a small amou...
6,Machine Learning,Distinct Types of ML,Reinforcement Learning,Reinforcement learning trains an agent to make...
...,...,...,...,...
49,Machine Learning,Tool,PyTorch,"Pythonic, intuitive, and integrates seamlessly..."
50,Machine Learning,Tool,PyTorch,"Great for experimentation, quick prototyping, ..."
51,Machine Learning,Tool,TensorFlow,"Stronger in production/industry, especially wi..."
52,Machine Learning,Tool,TensorFlow,"TensorFlow Serving, TensorFlow Lite (for mobil..."


In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

google_notes_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSQF2lNc4WPeTRQ_VzWPkqSZp4RODFkbap8AqmolWp5bKoMaslP2oRVVG21x2POu_JcbF1tGRcBgodu/pub?output=csv'
google_definition_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQq1-3cTas8DCWBa2NKYhVFXpl8kLaFDohg0zMfNTAU_Fiw6aIFLWfA5zRem4eSaGPa7UiQvkz05loW/pub?output=csv'

notes = pd.read_csv(google_notes_csv).fillna('')
definitions = pd.read_csv(google_definition_csv).fillna('')

ml_def = definitions.copy()
ml_notes = notes[notes['Category']=='Machine Learning']

from df_processing import notes_df_to_outline_html,final_dataset_for_markdown
d = final_dataset_for_markdown(ml_notes,ml_def)
d.to_csv('/Users/derekdewald/Documents/Python/Github_Repo/Streamlit/DataDictionary/d_learning_notes.csv',index=False)

d.head(25)

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
3,Machine Learning,Distinct Types of ML,Supervised Learning,Supervised learning trains a model using label...
4,Machine Learning,Distinct Types of ML,Unsupervised Learning,Unsupervised learning works with unlabeled dat...
5,Machine Learning,Distinct Types of ML,Semisupervised Learning,Semi-supervised learning combines a small amou...
6,Machine Learning,Distinct Types of ML,Reinforcement Learning,Reinforcement learning trains an agent to make...
1,Machine Learning,Optimization,Optimization,Optimization is the process of finding the bes...
7,Machine Learning,Optimization,Goal,"What are you attempting to maximize, Training ..."
8,Machine Learning,Optimization,Approach,Curse of Dimensionality
9,Machine Learning,Optimization,Important to Remember,Must remember what the Model is actually tryin...
10,Machine Learning,Optimization,Important to Remember,Models will cheat and can solve problems which...


In [3]:
d.head(45)

,Category,Categorization,Word,Notes
0,Machine Learning,Definition,Machine Learning,Using a process to enable computers to iterati...
3,Machine Learning,Distinct Types of ML,Supervised Learning,Supervised learning trains a model using label...
4,Machine Learning,Distinct Types of ML,Unsupervised Learning,Unsupervised learning works with unlabeled dat...
5,Machine Learning,Distinct Types of ML,Semisupervised Learning,Semi-supervised learning combines a small amou...
6,Machine Learning,Distinct Types of ML,Reinforcement Learning,Reinforcement learning trains an agent to make...
1,Machine Learning,Optimization,Optimization,Optimization is the process of finding the bes...
7,Machine Learning,Optimization,Goal,"What are you attempting to maximize, Training ..."
8,Machine Learning,Optimization,Approach,Curse of Dimensionality
9,Machine Learning,Optimization,Important to Remember,Must remember what the Model is actually tryin...
10,Machine Learning,Optimization,Important to Remember,Models will cheat and can solve problems which...


In [5]:
# Creates a Definition Object for Each Value
# If there is a Definition for Categorization it will include it.
d[d['Categorization']=='Validation']

,Category,Categorization,Word,Notes
2,Machine Learning,Validation,Validation,This is a Test Value. See if it works.
13,Machine Learning,Validation,Cross Validation,Model evaluation technique used to assess how ...
44,Machine Learning,Validation,Procedure,K Fold Cross Validation
45,Machine Learning,Validation,Procedure,Startified Cross Validation
46,Machine Learning,Validation,Procedure,Confusion Matrix
47,Machine Learning,Validation,Function,Accuracy


In [4]:
def final_dataset_for_markdown(notes=None,
                               definitions=None):
    '''
    Function which helps to combined Notes and Definitions into a Single Combined Representation which can ultimately be used as a Learning Reference Tools.


    
    '''

    from data_d_dicts import links

    try:
        len(notes)
    except:
        notes = pd.read_csv(links['google_notes_csv'])

    try:
        len(definitions)
    except:
        definitions = pd.read_csv(links['google_definition_csv'])

    merge_def = definitions[['Word','Definition','Category']]

    # For All Unique Categories, Add Definitions. Need to fill in other Columns of Dataframe for Future Merging
    temp_df = notes.drop_duplicates("Category")[['Category']]
    temp_df['Categorization'] = 'Definition'
    temp_df['Word'] = temp_df['Category']
    # Rank Utilized to provide Definitions with Highest Value when Categorizing, otherwise Sort a Issue when Merging
    temp_df['RANK'] = 0 
    
    # Merge In Combination of Category and Categorization
    temp_df1 = notes.drop_duplicates(['Category','Categorization']).drop(['Word','Notes'],axis=1).copy()
    temp_df1 = temp_df1.merge(definitions[['Word','Definition','Category']].rename(columns={'Word':'Categorization','Definition':"Notes"}),on=['Category','Categorization'],how='left')
    temp_df1 = temp_df1[temp_df1['Notes'].notnull()].copy()
    temp_df1['Word'] = temp_df1['Categorization'].copy()
    temp_df1['RANK'] = 0 
 
    # Merge In Definition to Category
    temp_df = temp_df.merge(merge_def.rename(columns={'Definition':"Notes"}),on=['Word','Category'],how='left')   
    temp_df2 = notes.merge(merge_def,on=['Word','Category'],how='left')
    temp_df2['RANK'] = 1

    # Merge In Listed Parameters 
    temp_df3 =  notes[['Category','Categorization','Word']].drop_duplicates(['Categorization','Word'])

    # Generate Information From Note
    list_ = definitions.drop('Notes',axis=1).rename(columns={'Word':'Notes','Categorization':'Word','Category':'Categorization'})[['Categorization','Word','Notes']]

    # Merge Into Notes.
    temp_df3 =  temp_df3.merge(list_,on=['Categorization','Word'],how='inner')

    return temp_df3
    # Create Final DataFrame
    final_df = pd.concat([temp_df,temp_df1,temp_df2,temp_df3])

    # Base Column is Notes, Definitions Merged in to Help Articulate and combine, thus if Notes Available, it will take precendent
    final_df['Notes'] = np.where(final_df['Notes']=="",final_df['Definition'],final_df['Notes'])
    final_df.drop('Definition',axis=1,inplace=True)
    
    # Calculuate Order DF. Using Order as defined in Notes Template.
    order_df =  notes[['Category','Categorization']].drop_duplicates().reset_index(drop=True).reset_index().rename(columns={'index':'ORDER'})
    final_df = final_df.merge(order_df,on=['Category','Categorization'],how='left')
    final_df['ORDER'] = final_df['ORDER'].fillna(0) 
    
    # Sort    
    final_df =  final_df.sort_values(['ORDER','RANK']).drop(['ORDER','RANK'],axis=1)

    # Remove Blank Place Holders that were satisfied by Merging Parameter Listing (Optimization, Optimizers as Example)
    final_df['count'] = final_df.groupby(['Category','Categorization','Word']).transform('size')
    final_df = final_df[~((final_df['count']>1)&(final_df['Notes'].isnull()))].copy()

    return final_df.drop('count',axis=1)


final_dataset_for_markdown(ml_notes,ml_def)

,Category,Categorization,Word,Notes
0,Machine Learning,Optimization,Algorithm,Adadelta
1,Machine Learning,Optimization,Algorithm,Adagrad
2,Machine Learning,Optimization,Algorithm,Adam
3,Machine Learning,Optimization,Algorithm,AdamW
4,Machine Learning,Optimization,Algorithm,Hinge Loss
5,Machine Learning,Optimization,Algorithm,Huber Loss
6,Machine Learning,Optimization,Algorithm,Momentum
7,Machine Learning,Optimization,Algorithm,Nadam
8,Machine Learning,Optimization,Algorithm,RMSprop
9,Machine Learning,Evaluation,Procedure,K Fold Cross Validation
